Prediction by PLS regression

Refference  
https://www.kaggle.com/pavelvpster/ion-switching-ridge

Previous commits

V9

windows = [50, 100, 1000, 5000, 10000, 25000]  
shift_range = 5  
test_size=0.50  
random_state = 0  
n_min = 1  
n_max=20  
n_splits = 5  

LB = 

V8

windows = [50, 100, 1000, 5000, 10000, 25000]  
shift_range = 15  
test_size=0.50  
random_state = 0  
n_min = 1  
n_max=20  
n_splits = 5

LB = 0.619

V4

windows = [50, 100, 1000, 5000, 10000, 25000]  
shift_range = 10  
test_size=0.50  
random_state = 0 
n_min = 1  
n_max=20  
n_splits = 5  

LB = 0.619



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from matplotlib import pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#read data
train = pd.read_csv('/kaggle/input/liverpool-ion-switching/train.csv')
test = pd.read_csv('/kaggle/input/liverpool-ion-switching/test.csv')
submission = pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv')

In [ ]:
#shift time data
train_time_0 = train['time'].values[:500000]
train_time_0 = list(train_time_0)*10
train['time'] = train_time_0

test_time_0 = train['time'].values[:500000]
test_time_0 = list(test_time_0)*4
test['time'] = test_time_0

In [ ]:
#add batch column
train['batch'] = 0
for i in range(0, 10):
    train.iloc[i * 500000: 500000 * (i + 1), 3] = i
test['batch'] = 0
for i in range(0, 4):
    test.iloc[i * 500000: 500000 * (i + 1), 2] = i

In [ ]:
def add_rolling_stats(data, windows):
    for window in windows:
        print('Window size:', window)
        stats = data['signal'].rolling(window=window)
        data["rolling_mean_" + str(window)] = stats.mean()
        data["rolling_std_"  + str(window)] = stats.std()
        data["rolling_min_"  + str(window)] = stats.min()
        data["rolling_max_"  + str(window)] = stats.max()
    data.fillna(0, inplace=True)

In [ ]:
#add rolling tatas columns
windows = [50, 100, 1000, 5000, 10000, 25000]
add_rolling_stats(train, windows)
add_rolling_stats(test, windows)

In [ ]:
def add_shifted_signal(data, shift):
    for batch in data['batch'].unique():
        m = data['batch'] == batch
        new_feature = 'shifted_signal_'
        if shift > 0:
            shifted_signal = np.concatenate((np.zeros(shift), data.loc[m, 'signal'].values[:-shift]))
            new_feature += str(shift)
        else:
            t = -shift
            shifted_signal = np.concatenate((data.loc[m, 'signal'].values[t:], np.zeros(t)))
            new_feature += 'minus_' + str(t)
        data.loc[m, new_feature] = shifted_signal

In [ ]:
#add shifted signal columns
shift_range = 5
for i in range(1,shift_range+1):
    add_shifted_signal(train, -i)
    add_shifted_signal(test, -i)
    add_shifted_signal(train, i)
    add_shifted_signal(test, i)

In [ ]:
y_train = train['open_channels'].copy()
x_train = train.drop(['time', 'batch', 'open_channels'], axis=1)

x_test = test.drop(['time', 'batch'], axis=1)

**PLS regression** 

In [ ]:
#import liblary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [ ]:
#auto scaling
x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape([-1,1]))
x_test_scaled = x_scaler.transform(x_test)

In [ ]:
#release memory
del x_train, x_test, train, test

In [ ]:
#Validation
x_train_train, x_train_valid, y_train_train, y_train_valid_scaled = train_test_split(x_train_scaled, y_train_scaled, test_size=0.50, random_state = 0)

In [ ]:
#search best n_components
n_min = 1
n_max=20
f1_arr = []
for i in range(n_min,n_max):
    pls = PLSRegression(n_components=i)
    pls.fit(x_train_train, y_train_train)
    y_pred = y_scaler.inverse_transform(pls.predict(x_train_valid))
    
    oof_part = np.clip(y_pred, 0, 10).astype('int')
    y_train_valid = y_scaler.inverse_transform(y_train_valid_scaled).astype('int')
    f1_arr.append(f1_score(y_train_valid,oof_part,average='macro'))
    print(i)

In [ ]:
plt.plot([x for x in range(n_min,n_max)],f1_arr,'o')
plt.xlabel('n_component')
plt.ylabel('Macro f1')
print(np.argmax(f1_arr) + n_min)#17

In [ ]:
n_splits = 5
n_components = np.argmax(f1_arr) + n_min
trained_estimators = []

cv = KFold(n_splits=n_splits, shuffle=True, random_state=0)
for train_idx, valid_idx in cv.split(x_train_scaled, y_train_scaled):
    
    x_train_train = x_train_scaled[train_idx]
    y_train_train = y_train_scaled[train_idx]
    x_train_valid = x_train_scaled[valid_idx]
    y_train_valid = y_train[valid_idx]
    
    estimator = PLSRegression(n_components=n_components)
    estimator.fit(x_train_train, y_train_train)
    trained_estimators.append(estimator)
    
    oof_part = y_scaler.inverse_transform(estimator.predict(x_train_valid))
    oof_part = np.clip(oof_part, 0, 10).astype('int')
    
    
    f1 = f1_score(y_train_valid.values,oof_part,average='macro')
    print('Macro F1:', f1)

In [ ]:
estimator

In [ ]:
y_preds = []
for estimator in trained_estimators:
    y_pred = y_scaler.inverse_transform(estimator.predict(x_test_scaled))
    y_preds.append(y_pred)
y_preds = np.stack(y_preds, axis=0)

y_preds.shape

In [ ]:
y_pred = np.mean(y_preds, axis=0)
y_pred = np.clip(y_pred, 0, 10).astype('int')

In [ ]:
plt.hist(y_pred)
plt.show()

In [ ]:
submission = pd.read_csv('../input/liverpool-ion-switching/sample_submission.csv')
submission['open_channels'] = y_pred
submission.to_csv('submission.csv', index=False, float_format='%.4f')

submission.head()